# Main analysis script

Put your leakpro results in the *data* folder using the following structure

data/main/*random_seed*/*dataset*/*target_model*/.../result.json

Example:

data/main/42/ELD/LSTM/LiRA-num_shadow_model=64-signal_name=RescaledSMAPELoss.../result.json


In [1]:
from utils import objects_to_ablations, aggregate_on_random_seed
import glob, os, json, gc

In [2]:
data_folder = "data/main"
output_folder = "output/main"

# This is the variable that varies and we want to observe the effect of
important_name = "signal_name"

# This is the parameters given in the structure of the data
from_path = ["random_seed", "dataset", "target_model"]

# The parameters given in config in result.json
from_config = ["online", "audit", "signal_name"]

def title_format(parameters):
    s = f'{parameters["dataset"]} ({parameters["target_model"]}) - {parameters["attack"]}'
    if "online" in parameters.keys():
        if parameters["online"]:
            s = s + " online"
        else:
            s = s + " offline"
    if "audit" in parameters.keys():
        if parameters["audit"]:
            s = s + " audit"
        else:
            s = s + " attack"
    s = s + " ROC Curve"
    return s

config = {
    "ds_indivs": {
        "EEG": 40,
        "ELD": 40,
    },
    "indiv_strategy": "indiv_mle", # indiv_mean, indiv_median, indiv_outlier, indiv_mle
    "order": ["MSELoss", "MAELoss", "SMAPELoss", "RescaledSMAPELoss", "TrendLoss", "SeasonalityLoss", "TS2VecLoss"],
    "title_format": title_format,
}


In [3]:
objects = []

for path in glob.glob(data_folder + "/**/*.json", recursive=True):
    # Add parameters from path
    data_folder_split = os.path.join(data_folder).split(os.sep)
    path_split = path.split(os.sep)[len(data_folder_split):]
    parameters = {param: path_split[i] for i, param in enumerate(from_path)}

    # Load json file
    with open(path) as f:
        data = json.load(f)

    # Add parameters from config in result.json
    parameters |= {k: v for k,v in data["config"].items() if k in from_config}
    if "signal_names" in data["config"].keys() and "signal_name" in from_config:
        parameters["signal_name"] = ",".join(data["config"]["signal_names"])

    # Assume attack name to be part of parameters
    parameters |= {"attack": data["result_name"]}
    
    if important_name not in parameters.keys():
        parameters[important_name] = parameters["attack"]

    important_value = parameters[important_name]
    parameters = {k: v for k,v in parameters.items() if k!=important_name}

    objects.append((data, parameters, important_value))
    print((parameters, important_value))


({'random_seed': '42', 'dataset': 'EEG', 'target_model': 'NHiTS', 'audit': True, 'attack': 'Ensemble'}, 'SeasonalityLoss')
({'random_seed': '42', 'dataset': 'EEG', 'target_model': 'NHiTS', 'online': True, 'attack': 'RMIA'}, 'MAELoss')
({'random_seed': '42', 'dataset': 'EEG', 'target_model': 'NHiTS', 'online': False, 'attack': 'RMIA'}, 'MAELoss')
({'random_seed': '42', 'dataset': 'EEG', 'target_model': 'NHiTS', 'online': True, 'attack': 'RMIA'}, 'SMAPELoss')
({'random_seed': '42', 'dataset': 'EEG', 'target_model': 'NHiTS', 'online': False, 'attack': 'RMIA'}, 'SeasonalityLoss')
({'random_seed': '42', 'dataset': 'EEG', 'target_model': 'NHiTS', 'audit': False, 'attack': 'Ensemble'}, 'TS2VecLoss')
({'random_seed': '42', 'dataset': 'EEG', 'target_model': 'NHiTS', 'audit': True, 'attack': 'Ensemble'}, 'RescaledSMAPELoss')
({'random_seed': '42', 'dataset': 'EEG', 'target_model': 'NHiTS', 'online': True, 'attack': 'LiRA'}, 'RescaledSMAPELoss')
({'random_seed': '42', 'dataset': 'EEG', 'target_mo

In [4]:
ablations = objects_to_ablations(objects, config)
del objects
gc.collect()

ablations = aggregate_on_random_seed(ablations) + ablations

In [5]:

for study in ablations:
    save_dir = f'{study.parameters["dataset"]}/{study.parameters["attack"]}/{study.parameters["target_model"]}'
    if "online" in study.parameters.keys():
        if study.parameters["online"]:
            save_dir = save_dir + "-online"
        else:
            save_dir = save_dir + "-offline"
    if "audit" in study.parameters.keys():
        if study.parameters["audit"]:
            save_dir = save_dir + "-audit_mode"
        else:
            save_dir = save_dir + "-attack_mode"
    
    save_dir = os.path.join(output_folder, save_dir, study.parameters["random_seed"])
    os.makedirs(save_dir, exist_ok=True) 
    study.make_table(save_dir)
    if not study.aggregated:
        study.make_roc_plot(save_dir)
        study.make_indiv_roc_plot(save_dir)
    gc.collect() # Garbage collect
    print(f"Saved study to {save_dir}")

Saved study to output/main/EEG/Ensemble/NHiTS-audit_mode/total
Saved study to output/main/EEG/RMIA/NHiTS-online/total
Saved study to output/main/EEG/RMIA/NHiTS-offline/total
Saved study to output/main/EEG/Ensemble/NHiTS-attack_mode/total
Saved study to output/main/EEG/LiRA/NHiTS-online/total
Saved study to output/main/EEG/LiRA/NHiTS-offline/total
Saved study to output/main/EEG/DTS-MIA/NHiTS-offline/total
Saved study to output/main/EEG/MS-LiRA/NHiTS-online/total
Saved study to output/main/EEG/MS-LiRA/NHiTS-offline/total
Saved study to output/main/EEG/DTS-MIA/NHiTS-online/total
Saved study to output/main/ELD/Ensemble/NHiTS-audit_mode/total
Saved study to output/main/ELD/RMIA/NHiTS-online/total
Saved study to output/main/ELD/RMIA/NHiTS-offline/total
Saved study to output/main/ELD/Ensemble/NHiTS-attack_mode/total
Saved study to output/main/ELD/LiRA/NHiTS-online/total
Saved study to output/main/ELD/LiRA/NHiTS-offline/total
Saved study to output/main/ELD/DTS-MIA/NHiTS-offline/total
Saved stu

<Figure size 640x480 with 0 Axes>